In [1]:
import os
import random
import math
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import special
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go


import torch 
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import grad

from torchvision import transforms
from torchsummary import summary
from visdom import Visdom

import sys
sys.path.append("../../modules/")
import helper as hp

device :  cuda
num gpu :  4


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = torch.device('cuda') 
print('device : ', device)
print('num gpu : ', torch.cuda.device_count())

device :  cuda
num gpu :  4


In [3]:
df = pd.read_csv("../data/trajectory.csv", low_memory=False).iloc[:100,:]

In [69]:
df_x = df.iloc[:, :5].astype(float)/360
df_traj = df.iloc[:, 5:8].astype(float)
df_col = np.heaviside(df.iloc[:, 8:].astype(int).sum(axis=1), 0).astype(int)

In [64]:
class FKDataset(data.Dataset):
    
    def __init__(self, df_x, df_y, traj=False, col=False):
        self.x = torch.tensor(df_x.to_numpy()).type(torch.float)
        if traj: self.y = torch.tensor(df_y.to_numpy()).type(torch.float)
        if col: self.y = df_y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        
        return x, y

In [12]:
model_fk = hp.BasicRegressor()
model_fk.layer1 = nn.Linear(5, 200)
model_fk.last = nn.Linear(200, 3)

In [13]:
dataset = FKDataset(df_x, df_traj, traj=True)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model_fk = hp.train(
    model_fk, dataloader,
    num_epochs=1, save_path=None)

-------------
Epoch 1/1
Loss:17.0634 
timer:  0.0736 sec.


In [70]:
model_col = hp.BasicRegressor()
model_col.layer1 = nn.Linear(5, 200)
model_col.last = nn.Linear(200, 2)

In [71]:
dataset = FKDataset(df_x, df_col, col=True)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model_col = hp.train(
    model_col, dataloader,
    criterion=nn.CrossEntropyLoss(), 
    num_epochs=1, save_path=None)

-------------
Epoch 1/1
Loss:0.5520 
timer:  0.0365 sec.
